In [ ]:
!pip install sqlalchemy psycopg2-binary

In [ ]:
import pandas as pd
import kagglehub
from kagglehub import KaggleDatasetAdapter

# from sqlalchemy import create_engine

# Uber NYC Dataset

In [ ]:
file_list = ["fhvhv_tripdata_2021-01.parquet"] # change to appropriate month

df_list = []

for file_name in file_list:
  df_list.append(
    kagglehub.load_dataset(
      KaggleDatasetAdapter.PANDAS,
      "shuhengmo/uber-nyc-forhire-vehicles-trip-data-2021",
      file_name,
    )
  )

uber_df = pd.concat(df_list)

# # Load the latest version
# df = kagglehub.load_dataset(
#   KaggleDatasetAdapter.PANDAS,
#   "shuhengmo/uber-nyc-forhire-vehicles-trip-data-2021",
#   file_path,
#   # Provide any additional arguments like
#   # sql_query or pandas_kwargs. See the
#   # documenation for more information:
#   # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
# )

# print("First 5 records:", df.head())

uber_df.head()

<ipython-input-3-7996e7ee5a41>:7: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  kagglehub.load_dataset(


100%|██████████| 391M/391M [00:07<00:00, 51.4MB/s]

Extracting zip of fhvhv_tripdata_2021-12.parquet...


,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B03404,B03404,2021-12-01 00:03:20,2021-12-01 00:07:14,2021-12-01 00:09:16,2021-12-01 00:58:03,164,265,17.54,...,6.42,2.75,0.0,0.00,53.67,N,N,,N,N
1,HV0003,B03404,B03404,2021-12-01 00:02:58,2021-12-01 00:05:38,2021-12-01 00:06:05,2021-12-01 00:19:05,80,112,2.39,...,1.18,0.00,0.0,0.00,9.41,N,N,,N,N
2,HV0003,B03404,B03404,2021-12-01 00:20:22,2021-12-01 00:21:26,2021-12-01 00:22:45,2021-12-01 00:43:47,112,189,4.91,...,2.00,0.00,0.0,5.05,16.23,N,N,,N,N
3,HV0003,B03404,B03404,2021-12-01 00:47:40,2021-12-01 00:48:50,2021-12-01 00:50:51,2021-12-01 00:59:20,49,225,1.59,...,0.85,0.00,0.0,0.00,6.26,N,N,,N,N
4,HV0003,B03404,B03404,2021-12-01 00:25:31,2021-12-01 00:29:07,2021-12-01 00:29:12,2021-12-01 00:37:22,239,263,1.78,...,0.63,2.75,0.0,7.00,6.07,N,N,,N,N


In [ ]:
uber_df.shape

(12086389, 11)

In [ ]:
# uber_df[['request_datetime', 'pickup_datetime', 'dropoff_datetime', 'dropoff_datetime', 'trip_miles', 'trip_time', 'base_passenger_fare', 'tips']].describe()

In [ ]:
uber_df.dtypes

,0
tolls,float64
trip_miles,float64
PULocationID,int64
trip_time,int64
on_scene_datetime,datetime64[us]
request_datetime,datetime64[us]
DOLocationID,int64
driver_pay,float64
tips,float64
total_fare,float64


In [ ]:
# isolate only Uber rides
uber_df = uber_df[uber_df['hvfhs_license_num'] == 'HV0003']

# request hour floor used to merge with weather_df
uber_df['request_hour'] = uber_df['request_datetime'].dt.floor('H')

# replace null airport fee with 0
uber_df['airport_fee'].fillna(0, inplace=True)

# calculate total_fare excluding tolls and tips
uber_df['total_fare'] = uber_df['base_passenger_fare'] + uber_df['sales_tax'] + uber_df['congestion_surcharge'] + uber_df['airport_fee']

# isolate specific columns of interest
uber_df = uber_df[['tolls', 'trip_miles', 'PULocationID', 'trip_time', 'on_scene_datetime', 'request_datetime', 'DOLocationID', 'driver_pay', 'tips', 'total_fare', 'request_hour']]

<ipython-input-4-e860293850ba>:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  uber_df['request_hour'] = uber_df['request_datetime'].dt.floor('H')
<ipython-input-4-e860293850ba>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  uber_df['airport_fee'].fillna(0, inplace=True)


In [ ]:
uber_df = uber_df.reset_index(drop=True)
uber_df['ride_id'] = uber_df.index + 1

In [ ]:
uber_df.isnull().any()

,0
tolls,False
trip_miles,False
PULocationID,False
trip_time,False
on_scene_datetime,False
request_datetime,False
DOLocationID,False
driver_pay,False
tips,False
total_fare,False


In [ ]:
uber_df

,tolls,trip_miles,PULocationID,trip_time,on_scene_datetime,request_datetime,DOLocationID,driver_pay,tips,total_fare,request_hour
0,0.0,17.54,164,2928,2021-12-01 00:07:14,2021-12-01 00:03:20,265,53.67,0.00,81.49,2021-12-01 00:00:00
1,0.0,2.39,80,780,2021-12-01 00:05:38,2021-12-01 00:02:58,112,9.41,0.00,14.45,2021-12-01 00:00:00
2,0.0,4.91,112,1262,2021-12-01 00:21:26,2021-12-01 00:20:22,189,16.23,5.05,24.59,2021-12-01 00:00:00
3,0.0,1.59,49,509,2021-12-01 00:48:50,2021-12-01 00:47:40,225,6.26,0.00,10.43,2021-12-01 00:00:00
4,0.0,1.78,239,490,2021-12-01 00:29:07,2021-12-01 00:25:31,263,6.07,7.00,12.53,2021-12-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...
16054485,0.0,3.27,50,1146,2021-12-31 23:38:02,2021-12-31 23:35:41,79,14.66,0.00,24.01,2021-12-31 23:00:00
16054486,0.0,2.79,161,3112,2021-12-31 23:08:20,2021-12-31 22:59:35,48,32.95,0.00,23.45,2021-12-31 22:00:00
16054487,0.0,3.69,167,656,2021-12-31 23:41:04,2021-12-31 23:39:38,235,10.42,0.00,16.31,2021-12-31 23:00:00
16054488,0.0,3.02,79,1003,2021-12-31 23:24:34,2021-12-31 23:21:52,87,13.59,0.00,16.98,2021-12-31 23:00:00


In [ ]:
from google.colab import files

uber_df.to_csv('uber_rides_dec.csv', index=False)

files.download('uber_rides_dec.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# DB_HOST = 'nyc-ride-share.cr1qs8bpfelg.us-east-1.rds.amazonaws.com'
# DB_USER = 'postgres'
# DB_PW = 'qEVEuU44rN6snSdQId6c'
# DB_PORT = '5432'
# DB_NAME = 'uber_ride_share'

# engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PW}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

In [ ]:
# chunk_size = 100000

# uber_df.iloc[:chunk_size].to_sql('uber_rides', engine, if_exists='replace', index=False)

# for i in range(chunk_size, len(uber_df), chunk_size):
#     chunk = uber_df.iloc[i:i+chunk_size]
#     chunk.to_sql('uber_rides', engine, if_exists='append', index=False)

# NYC Weather Dataset

In [ ]:
weather_df = pd.read_csv('weather_data.csv')
weather_df.head()

,time,temperature_2m (°C),apparent_temperature (°C),precipitation (mm),rain (mm),snowfall (cm),snow_depth (m),wind_speed_10m (km/h),wind_gusts_10m (km/h)
0,2021-01-01T00:00,3.6,-1.3,0.0,0.0,0.0,0.0,14.6,26.3
1,2021-01-01T01:00,3.4,-1.4,0.0,0.0,0.0,0.0,13.6,24.1
2,2021-01-01T02:00,2.8,-2.0,0.0,0.0,0.0,0.0,13.7,22.7
3,2021-01-01T03:00,0.8,-3.6,0.0,0.0,0.0,0.0,10.6,22.3
4,2021-01-01T04:00,0.0,-4.5,0.0,0.0,0.0,0.0,10.8,18.7


In [ ]:
weather_df.dtypes

,0
time,datetime64[ns]
temperature_2m (°C),float64
apparent_temperature (°C),float64
precipitation (mm),float64
rain (mm),float64
snowfall (cm),float64
snow_depth (m),float64
wind_speed_10m (km/h),float64


In [ ]:
weather_df.shape

(8760, 9)

In [ ]:
weather_df.isnull().any()

,0
time,False
temperature_2m (°C),False
apparent_temperature (°C),False
precipitation (mm),False
rain (mm),False
snowfall (cm),False
snow_depth (m),False
wind_speed_10m (km/h),False
wind_gusts_10m (km/h),False


In [ ]:
# weather_df.describe()

In [ ]:
weather_df['time'] = pd.to_datetime(weather_df['time'])

weather_df = weather_df.drop(columns=['wind_gusts_10m (km/h)'])

In [ ]:
weather_df = weather_df.rename(columns={
    'temperature_2m (°C)': 'temperature',
    'apparent_temperature (°C)': 'apparent_temperature',
    'precipitation (mm)': 'precipitation',
    'rain (mm)': 'rain',
    'snowfall (cm)': 'snowfall',
    'snow_depth (m)': 'snow_depth',
    'wind_speed_10m (km/h)': 'wind_speed'
})

In [ ]:
weather_df.head()

,time,temperature,apparent_temperature,precipitation,rain,snowfall,snow_depth,wind_speed
0,2021-01-01 00:00:00,3.6,-1.3,0.0,0.0,0.0,0.0,14.6
1,2021-01-01 01:00:00,3.4,-1.4,0.0,0.0,0.0,0.0,13.6
2,2021-01-01 02:00:00,2.8,-2.0,0.0,0.0,0.0,0.0,13.7
3,2021-01-01 03:00:00,0.8,-3.6,0.0,0.0,0.0,0.0,10.6
4,2021-01-01 04:00:00,0.0,-4.5,0.0,0.0,0.0,0.0,10.8


In [ ]:
from google.colab import files

weather_df.to_csv('weather_cleaned.csv', index=False)

files.download('weather_cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Merge Dataframes

In [ ]:
merged_df = pd.merge(uber_df, weather_df, left_on='request_hour', right_on='time')


In [ ]:
merged_df = merged_df.drop(columns=['time'])

In [ ]:
merged_df

,tolls,trip_miles,PULocationID,trip_time,on_scene_datetime,request_datetime,DOLocationID,driver_pay,tips,total_fare,request_hour,temperature_2m (°C),apparent_temperature (°C),precipitation (mm),rain (mm),snowfall (cm),snow_depth (m),wind_speed_10m (km/h)
0,0.0,5.26,230,923,2021-01-01 00:31:42,2021-01-01 00:28:09,166,14.99,0.00,27.01,2021-01-01 00:00:00,3.6,-1.3,0.0,0.0,0.00,0.00,14.6
1,0.0,3.65,152,1382,2021-01-01 00:55:19,2021-01-01 00:45:56,167,17.06,0.00,19.99,2021-01-01 00:00:00,3.6,-1.3,0.0,0.0,0.00,0.00,14.6
2,0.0,3.51,233,849,2021-01-01 00:22:41,2021-01-01 00:21:15,142,12.98,0.94,18.05,2021-01-01 00:00:00,3.6,-1.3,0.0,0.0,0.00,0.00,14.6
3,0.0,0.74,142,179,2021-01-01 00:42:37,2021-01-01 00:39:12,143,7.41,0.00,11.36,2021-01-01 00:00:00,3.6,-1.3,0.0,0.0,0.00,0.00,14.6
4,0.0,9.20,143,1228,2021-01-01 00:47:17,2021-01-01 00:46:11,78,22.44,0.00,32.27,2021-01-01 00:00:00,3.6,-1.3,0.0,0.0,0.00,0.00,14.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8703789,0.0,2.29,89,870,2021-01-31 23:25:03,2021-01-31 23:13:51,71,12.06,0.00,18.27,2021-01-31 23:00:00,-4.0,-9.4,0.2,0.1,0.07,0.02,14.6
8703790,0.0,4.10,182,1093,2021-01-31 23:29:03,2021-01-31 23:23:56,167,13.66,2.00,18.72,2021-01-31 23:00:00,-4.0,-9.4,0.2,0.1,0.07,0.02,14.6
8703791,0.0,2.40,167,904,2021-01-31 23:49:23,2021-01-31 23:42:53,169,15.53,0.00,21.26,2021-01-31 23:00:00,-4.0,-9.4,0.2,0.1,0.07,0.02,14.6
8703792,0.0,3.60,188,1097,2021-01-31 23:09:13,2021-01-31 23:04:32,37,17.36,0.00,21.63,2021-01-31 23:00:00,-4.0,-9.4,0.2,0.1,0.07,0.02,14.6


In [ ]:
merged_df.dtypes

,0
tolls,float64
trip_miles,float64
PULocationID,int64
trip_time,int64
on_scene_datetime,datetime64[us]
request_datetime,datetime64[us]
DOLocationID,int64
driver_pay,float64
tips,float64
total_fare,float64


In [ ]:
merged_df.shape

(8703794, 18)

In [ ]:
merged_df.isnull().any()

,0
tolls,False
trip_miles,False
PULocationID,False
trip_time,False
on_scene_datetime,False
request_datetime,False
DOLocationID,False
driver_pay,False
tips,False
total_fare,False


In [ ]:
null_mask = merged_df.isnull().any(axis=1)
null_rows_df = merged_df[null_mask]

null_rows_df

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,wav_match_flag,request_hour,temperature_2m (°C),apparent_temperature (°C),precipitation (mm),rain (mm),snowfall (cm),snow_depth (m),wind_speed_10m (km/h),wind_gusts_10m (km/h)
0,HV0003,B02682,B02682,2021-01-01 00:28:09,2021-01-01 00:31:42,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,5.26,...,N,2021-01-01 00:00:00,3.6,-1.3,0.0,0.0,0.00,0.00,14.6,26.3
1,HV0003,B02682,B02682,2021-01-01 00:45:56,2021-01-01 00:55:19,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,3.65,...,N,2021-01-01 00:00:00,3.6,-1.3,0.0,0.0,0.00,0.00,14.6,26.3
2,HV0003,B02764,B02764,2021-01-01 00:21:15,2021-01-01 00:22:41,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,3.51,...,N,2021-01-01 00:00:00,3.6,-1.3,0.0,0.0,0.00,0.00,14.6,26.3
3,HV0003,B02764,B02764,2021-01-01 00:39:12,2021-01-01 00:42:37,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,0.74,...,N,2021-01-01 00:00:00,3.6,-1.3,0.0,0.0,0.00,0.00,14.6,26.3
4,HV0003,B02764,B02764,2021-01-01 00:46:11,2021-01-01 00:47:17,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,9.20,...,N,2021-01-01 00:00:00,3.6,-1.3,0.0,0.0,0.00,0.00,14.6,26.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8703789,HV0003,B02765,B02765,2021-01-31 23:13:51,2021-01-31 23:25:03,2021-01-31 23:25:40,2021-01-31 23:40:10,89,71,2.29,...,N,2021-01-31 23:00:00,-4.0,-9.4,0.2,0.1,0.07,0.02,14.6,24.8
8703790,HV0003,B02872,B02872,2021-01-31 23:23:56,2021-01-31 23:29:03,2021-01-31 23:29:31,2021-01-31 23:47:44,182,167,4.10,...,N,2021-01-31 23:00:00,-4.0,-9.4,0.2,0.1,0.07,0.02,14.6,24.8
8703791,HV0003,B02872,B02872,2021-01-31 23:42:53,2021-01-31 23:49:23,2021-01-31 23:49:32,2021-02-01 00:04:36,167,169,2.40,...,N,2021-01-31 23:00:00,-4.0,-9.4,0.2,0.1,0.07,0.02,14.6,24.8
8703792,HV0003,B02764,B02764,2021-01-31 23:04:32,2021-01-31 23:09:13,2021-01-31 23:09:29,2021-01-31 23:27:46,188,37,3.60,...,N,2021-01-31 23:00:00,-4.0,-9.4,0.2,0.1,0.07,0.02,14.6,24.8
